In [3]:
# Python3 code to demonstrate working of
# Functions as dictionary values
# Using With params

# call Gfg fnc
def sum_key(a, b):
  return a + b

# initializing dictionary
# check for function name as key
test_dict = {"Gfg": sum_key, "is" : 5, "best" : 9}

# printing original dictionary
print("The original dictionary is : " + str(test_dict))

# calling function using brackets
# params inside brackets
res = test_dict['Gfg'](10, 34)

# printing result
print("The required call result : " + str(res))


The original dictionary is : {'Gfg': <function sum_key at 0x7fc5a5105fc0>, 'is': 5, 'best': 9}
The required call result : 44


In [4]:
## SETUP ##

import ccxt
from Values import BYBIT_API_KEY, BYBIT_API_SECRET, OHLCV_DIR
import Utilities as utils
import pandas as pd
import warnings
from pybit import usdt_perpetual

pybit_auth = usdt_perpetual.HTTP(
    endpoint="https://api.bybit.com",
    api_key=BYBIT_API_KEY,
    api_secret=BYBIT_API_SECRET
)

# Switch Logging off and on
debugLog = True

warnings.simplefilter(action='ignore')
pd.set_option('display.max_rows', None)

bybit = ccxt.bybit({'apiKey': BYBIT_API_KEY, 'secret': BYBIT_API_SECRET})
data_dir = OHLCV_DIR


In [5]:
## Test Data ##
base_cur = "USDT"
exch_cur = "ETH"
symbol = '_'.join([exch_cur, base_cur])
timeframe = "5m"
maxCandles = 500
runUpCandles = 20
# Dataframe from test json file (backtesting)
jsonfile = f"{data_dir}{symbol}-{timeframe}.json"
print(jsonfile)
df = pd.read_json(jsonfile)
df.head()
#df = df[-500:-2]
#df.columns = ['timestamp','open','high','low','close','volume']
#df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
#df['trade'] = 'no_trade'
#trades = pd.DataFrame(columns=['time', 'leverage', 'fees', 'side', 'stake', 'entry_price', 'exit_price', 'percent_difference', 'PNL', 'entry_reason', 'exit_reason', 'entry_balance', 'exit_balance', 'status'])
#superTrend(df, multiplier=5, period=9)
#df_backup = df.copy()


../../data/bybit/ETH_USDT-5m.json


,0,1,2,3,4,5
0,1616716800000,1588.45,1604.35,1588.05,1604.35,310.05
1,1616717100000,1604.35,1606.35,1601.25,1605.00,424.53
2,1616717400000,1605.00,1609.10,1605.00,1606.05,95.67
3,1616717700000,1606.05,1610.85,1602.40,1607.85,196.46
4,1616718000000,1607.85,1611.35,1606.20,1609.00,99.12


In [6]:
## FUNCTIONS ##

# Get Balance fetch_balance dict_keys(['info', 'ADA', 'BIT', 'BTC', 'DOT', 'EOS',
#                                      'ETH', 'LTC', 'LUNA', 'MANA', 'SOL', 'USDT',
#                                      'XRP', 'free', 'used', 'total'])
def GetTotalBalance(verbose=debugLog) -> dict:
    full_balances = bybit.fetch_balance()
    totals = full_balances['total']
    balance = {}
    for t in totals:
        if totals[t] > 0:
            balance[str(t)] = totals[t]
    return balance
#print(GetTotalBalance())
def GetFreeBalance(verbose=debugLog) -> dict:
    full_balances = bybit.fetch_balance()
    totals = full_balances['free']
    balance = {}
    for t in totals:
        if totals[t] > 0:
            balance[str(t)] = totals[t]
    return balance
#print(GetFreeBalance())
def GetUsedBalance(verbose=debugLog) -> dict:
    full_balances = bybit.fetch_balance()
    totals = full_balances['used']
    balance = {}
    for t in totals:
        if totals[t] > 0:
            balance[str(t)] = totals[t]
    return balance
#print(GetUsedBalance())

# Get all symbol ids, or all of a given quote currency
def GetSymbols(quote_currency="", verbose=debugLog) -> list:
    marketsFull = bybit.fetch_markets()
    symbols = []
    count = 0
    if quote_currency == "":
        for m in marketsFull:
            symbols.append(m['id'])
    else:
        for m in marketsFull:
            if m['quote'] == quote_currency:
                symbols.append(m['id'])
    if verbose: print(f"count: {len(symbols)}")
    return symbols
#print(GetSymbols('USDT'))

def GetTop24hVol(number: int=0, stripped: bool=True, verbose: bool=debugLog) -> tuple:
    if number > 99:
        if verbose: print("Please pick a smaller number")
        return ()
    tickers = bybit.fetch_tickers()
    all_tickers_sym = tickers.keys()
    vols = {}
    for key in all_tickers_sym:
        tick = tickers[key]
        if  tick['info']['symbol'][-4:] == 'USDT':
            vol = float(tick['quoteVolume']) * tick['last']
            vols[vol] = key
    sorted_vol = sorted(vols.items())
    sorted_vol = list(reversed(sorted_vol))
    if number > 0: sorted_vol = sorted_vol[:number]
    if stripped:
        stripped_list = []
        for res in sorted_vol:
            # BTC/USDT:USDT -> BTCUSDT
            stripped_list.append("".join(res[1].split(':')[0].split('/')))
        sorted_vol = stripped_list
    return tuple(sorted_vol)
#print(GetTop24hVol(20, True, True))

# Create a Simple Buy/Sell Market Order
# amount - size of position in base currency
def CreateSimpleMarketOrder(sym: str = 'ADAUSDT', side: str = 'buy', amount: float = 1.0, verbose=debugLog) -> dict:
    return bybit.create_market_order(sym, side, amount, 0)
#print(CreateSimpleMarketOrder('XRPUSDT'))

# Create a Simple Buy/Sell Limit Order
def CreateSimpleLimitOrder(sym: str = 'ADAUSDT', side: str = 'buy', amount: float = 1.0, price: float = 0.48, verbose=debugLog) -> dict:
    return bybit.create_limit_order(sym, side, amount, price)
#print(CreateSimpleLimitOrder('XRPUSDT', amount=10))

# TODO #
########
def CreatSimpleConditionalOrder():
    return bybit.create_contract_order()

# Get open positions
def GetOpenPositions() -> dict:
    my_positions = pybit_auth.my_position()['result']
    positions = {}
    for p in my_positions:
        if p['data']['size'] > 0: positions[p['data']['symbol']] = p['data']
    return positions
#print(GetOpenPositions())

# Get All active limit orders
def GetActiveOrders(symbol_list: list = []) -> list:
    limits = []
    # default list
    if symbol_list == []: symbol_list =  list(GetTop24hVol(20, True, False))
    for sym in symbol_list:
        my_order = pybit_auth.get_active_order(symbol=sym)['result']['data']
        #print(my_order)
        if my_order != None:
            for ord in my_order:
                if ord['order_status'] != 'Filled' and ord['order_status'] != 'Cancelled': limits.append(ord)
    return limits
#print(GetActiveOrders())

# Get All active conditional orders
def GetConditionalOrders(symbol_list: list = []) -> list:
    conditionals = []
    # default list
    if symbol_list == []: symbol_list = list(GetTop24hVol(20, True, False))
    for sym in symbol_list:
        my_order = pybit_auth.get_conditional_order(symbol=sym)['result']['data']
        #print(my_order)
        if my_order != None:
            for ord in my_order:
                if ord['order_status'] != 'Filled' and ord['order_status'] != 'Deactivated': conditionals.append(ord)
    return conditionals
#print(GetConditionalOrders())
